# Speech separation CNN model

In this notebook we implemented an U-net shaped convolutional network loosely based on Yu et al 2017.

### Basic setup

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


The shape parameters:

In [0]:
S = 2   # Number of speakers
F = 257 # Number of STFT frequency bins
T = 431 # Number of STFT time bins
SFT=S*F*T

Using our previously generated dataset, which is in HDF5 format:

In [0]:
h5Path = "/content/gdrive/My Drive/Nagy házi/audiobooks/train_data/data2.hdf5"

In [0]:
%tensorflow_version 1.x
from tensorflow.keras.utils import HDF5Matrix

In [0]:
x_train = HDF5Matrix(datapath=h5Path,
                     #normalizer=normalizeInput,
                     dataset='trainInput')

y_train = HDF5Matrix(datapath=h5Path,
                     dataset='trainOutput')

x_valid = HDF5Matrix(datapath=h5Path,
                     #normalizer=normalizeInput,
                     dataset='validInput')

y_valid = HDF5Matrix(datapath=h5Path,
                     dataset='validOutput')

Due to a [bug](https://github.com/tensorflow/tensorflow/issues/30993) in Tensorflow, the validation data cannot be in HDF5 format:

In [0]:
import numpy as np
x_valid_arr = np.array(x_valid)
y_valid_arr = np.array(y_valid)

The input of the network is the STFT magnitude of the mixed speech.

In [0]:
input_shape = (F,T,1)

The output are two masks that are used to isolate the magnitude spectra corresponding to the two speakers.

In [0]:
output_shape = (F,T,2)

## Modell

Now we define the model:

In [0]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Reshape, Flatten, UpSampling2D, Cropping2D, ZeroPadding2D, Lambda, BatchNormalization
from tensorflow.keras.activations import softmax
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.initializers import RandomUniform
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np

A softmax activation was used in Yu et al 2017 on the last axis of the output:

In [0]:
def softMaxAxisLast(x):
    return softmax(x,axis=-1)

In [12]:
model = Sequential()
# 1 x (1,64)-(2,2)
model.add(BatchNormalization(input_shape=input_shape))
model.add(Conv2D(filters=64,
                 strides=(2,2),
                 kernel_size=(3,3),
                 activation='relu',
                 padding='same'
                 ))
# 4 x (64,64) - (1,1)
'''
model.add(Conv2D(filters=64,
                 strides=(1,1),
                 kernel_size=(3,3),
                 padding='same',
                 activation='relu'
                 ))
model.add(Conv2D(filters=64,
                 strides=(1,1),
                 kernel_size=(3,3),
                 padding='same',
                 activation='relu'
                 ))
model.add(Conv2D(filters=64,
                 strides=(1,1),
                 kernel_size=(3,3),
                 padding='same',
                 activation='relu'
                 ))
model.add(Conv2D(filters=64,
                 strides=(1,1),
                 kernel_size=(3,3),
                 padding='same',
                 activation='relu'
                 ))
'''
# 1 x (64,128) - (2,2)
model.add(Conv2D(filters=128,
                 strides=(2,2),
                 kernel_size=(3,3),
                 padding='same',
                 activation='relu'
                 ))
# 2 x (128,128) - (1,1)
'''
model.add(Conv2D(filters=128,
                 strides=(1,1),
                 kernel_size=(3,3),
                 padding='same',
                 activation='relu'
                 ))
model.add(Conv2D(filters=128,
                 strides=(1,1),
                 kernel_size=(3,3),
                 padding='same',
                 activation='relu'
                 ))
'''
# 1 x (128,256) - (2,2)
model.add(Conv2D(filters=256,
                 strides=(2,2),
                 kernel_size=(3,3),
                 padding='same',
                 activation='relu'
                 ))
# 2 x (256,256) - (1,1)
model.add(Conv2D(filters=256,
                 strides=(1,1),
                 kernel_size=(3,3),
                 padding='same',
                 activation='relu'
                 ))
'''
model.add(Conv2D(filters=256,
                 strides=(1,1),
                 kernel_size=(3,3),
                 padding='same',
                 activation='relu'
                 ))
'''

model.add(MaxPooling2D(2,2))

model.add(Conv2D(filters=256,
                 strides=(1,1),
                 kernel_size=(3,3),
                 padding='same',
                 activation='relu'
                 ))

model.add(UpSampling2D(size=(2, 2)))

model.add(Conv2D(filters=128,
                 strides=(1,1),
                 kernel_size=(3,3),
                 padding='same',
                 activation='relu'
                 ))

model.add(UpSampling2D(size=(2, 2)))

model.add(Conv2D(filters=32,
                 strides=(1,1),
                 kernel_size=(3,3),
                 padding='same',
                 activation='relu'
                 ))

model.add(UpSampling2D(size=(2, 2)))

model.add(Conv2D(filters=8,
                 strides=(1,1),
                 kernel_size=(3,3),
                 padding='same',
                 activation='relu'
                 ))

model.add(UpSampling2D(size=(2, 2)))

model.add(Conv2D(filters=4,
                 strides=(1,1),
                 kernel_size=(3,3),
                 padding='same',
                 activation='relu'
                 ))

model.add(ZeroPadding2D(padding=(1,0)))

model.add(Conv2D(filters=2,
                 strides=(1,1),
                 kernel_size=(2,2),
                 activation='relu'
                 ))

#model.add(Activation(softMaxAxisLast))

'''
model.add(Lambda(lambda x: K.spatial_2d_padding(x,
                                                padding=((0,0),(0,1)),
                                                data_format='channels_first'
                                                )))
'''

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


"\nmodel.add(Lambda(lambda x: K.spatial_2d_padding(x,\n                                                padding=((0,0),(0,1)),\n                                                data_format='channels_first'\n                                                )))\n"

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 257, 431, 1)       4         
_________________________________________________________________
conv2d (Conv2D)              (None, 129, 216, 64)      640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 65, 108, 128)      73856     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 33, 54, 256)       295168    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 33, 54, 256)       590080    
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 27, 256)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 27, 256)       5

## Loss

The main idea of Yu et al. 2017 was permutation invariant training. After the masks output by the network are applied to the STFT amplitude of the mixed signal, the speaker assignment is chosen from the permutation yielding the least error.

For this we needed to define a custom loss function, which is implemented for Keras in the `pitLossWrapper` function.

In [0]:
def yuxuanError(estXsAbs, XsAbs):
  #err = 0
  #for i in range(S):
    #err = err + np.linalg.norm()**2
  err0 = K.sum(K.abs(estXsAbs[0] - XsAbs[...,0])**2) # ide kéne négyzet
  err1 = K.sum(K.abs(estXsAbs[1] - XsAbs[...,1])**2) # ide kéne négyzet
  err = err0 + err1
  return err/SFT

import itertools
def pitLoss(estXsAbs, XsAbs):
  return K.min([yuxuanError(list(p), XsAbs) for p in itertools.permutations(estXsAbs)])

def pitLossWrapper(y_true, y_pred):
  estMs = y_pred[...,:2]
  XsAbs = y_true[...,1:]
  Yabs = y_true[...,0]
  estXsAbs = [estMs[...,0] * Yabs, estMs[...,1] * Yabs]
  #estXsAbs = [y_pred[...,1], y_pred[...,2]]
  return pitLoss(estXsAbs, XsAbs)

In [15]:
testOutput = model.predict(np.expand_dims(x_train[7], 0))
testOutput[0][...,0]

array([[9.5124273e-03, 7.5888182e-03, 1.8718837e-02, ..., 1.3656422e-02,
        1.1709627e-02, 5.8137241e-04],
       [1.7179776e-02, 1.8097755e-02, 2.0716665e-02, ..., 0.0000000e+00,
        1.0805569e-04, 2.2001450e-03],
       [9.1532413e-03, 3.9381245e-03, 3.8624180e-03, ..., 4.6544592e-03,
        8.2597490e-03, 9.4682425e-03],
       ...,
       [1.0177229e-05, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [9.2936862e-06, 4.1350377e-06, 7.3150636e-06, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        2.7994336e-08, 2.7405326e-07]], dtype=float32)

## Training

For the first try, we chose the ADAM optimizer.

In [0]:
model.compile(optimizer='adam',
              loss=pitLossWrapper)

Early stopping:

In [0]:
early_stopping = EarlyStopping(monitor='val_loss',
                               min_delta=0,
                               patience=5,
                               verbose=0,
                               mode='auto')

checkpoint = ModelCheckpoint('/content/gdrive/My Drive/Nagy házi/es_weights.hdf5',
                             monitor='val_loss',
                             save_best_only=True,
                             save_weights_only=True)

Fitting the model:

In [18]:
model.fit(x=x_train,
          y=y_train,
          validation_data=(x_valid_arr,y_valid_arr),
          epochs=50,
          shuffle='batch',
          callbacks=[early_stopping, checkpoint],
          verbose=1)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 4455 samples, validate on 247 samples
Epoch 1/50
4455/4455 [==============================] - 124s 28ms/sample - loss: 8.9483 - val_loss: 8.0659
Epoch 2/50
4455/4455 [==============================] - 105s 23ms/sample - loss: 8.2145 - val_loss: 7.9100
Epoch 3/50
4455/4455 [==============================] - 104s 23ms/sample - loss: 7.1184 - val_loss: 5.8631
Epoch 4/50
4455/4455 [==============================] - 103s 23ms/sample - loss: 4.9712 - val_loss: 4.9114
Epoch 5/50
4455/4455 [==============================] - 104s 23ms/sample - loss: 3.8016 - val_loss: 3.5252
Epoch 6/50
4455/4455 [==============================] - 103s 23ms/sample - loss: 3.1861 - val_loss: 3.2592
Epoch 7/50
4455/4455 [==============================] - 104s 23ms/sample - loss: 2.7826 - val_loss: 3.0619
Epoch 8/50
4455/4455 [==============================] - 104s 23ms/sample - loss: 2.5395 - val_loss: 2.9231
Epo

KeyboardInterrupt: ignored

The CNN learns, but the loss could be better. We will try to improve the performance for the final milestone. However, it already produces some usable output.

## Proof of concept

### Getting some example data:

In [0]:
import librosa
import IPython.display
import sys
import numpy as np
sys.path.append('/content/gdrive/My Drive/Nagy házi/')
from audio_masking import *

Load two arbitrarily selected utterances:

In [0]:
files = ['/content/gdrive/My Drive/Nagy házi/audiobooks/single_stft/hardtimes_04_dickens__Joseph_Ugoretz_11.npy',
 '/content/gdrive/My Drive/Nagy házi/audiobooks/single_stft/hardtimes_03_dickens__Rosalind_Wills_191.npy']

In [0]:
X1 = np.load(files[0], allow_pickle=False, fix_imports=True)
X2 = np.load(files[1], allow_pickle=False, fix_imports=True)

Mix the two streams, and calculate the spectrum magnitude to be fed into the model:

In [0]:
Y = X1+X2
YAbs = np.abs(Y)

The audio stream of the first speaker:

In [23]:
IPython.display.Audio(data=librosa.istft((X1), hop_length=256), rate=22050)

The audio stream of the second speaker:

In [24]:
IPython.display.Audio(data=librosa.istft((X2), hop_length=256), rate=22050)

The mixed audio:

In [25]:
IPython.display.Audio(data=librosa.istft((Y), hop_length=256), rate=22050)

### Speech separation

Generate two masks using the model:

In [0]:
testMs = model.predict(np.expand_dims(np.atleast_3d(YAbs), 0))

Their sum should be close to one:

In [29]:
testMs[0][...,0]+testMs[0][...,1]

array([[0.7627641 , 0.743112  , 0.8847914 , ..., 0.986899  , 1.0812514 ,
        0.79930675],
       [0.8984343 , 1.0779991 , 1.1653956 , ..., 1.1305099 , 1.0654459 ,
        1.0279627 ],
       [0.90480095, 1.1296399 , 1.1498481 , ..., 1.0459031 , 1.0221009 ,
        1.0508568 ],
       ...,
       [0.72083175, 0.7498288 , 0.81916785, ..., 0.7518356 , 0.69799626,
        0.77585447],
       [0.5216606 , 0.5977382 , 0.6297992 , ..., 0.6697935 , 0.64851207,
        0.6828207 ],
       [0.3400699 , 0.28161618, 0.29497537, ..., 0.3027594 , 0.34217006,
        0.34378588]], dtype=float32)

Reconstruct the whole STFT spectrum for each speaker:

In [0]:
[X1test, X2test] = reconstructSpectrum([testMs[0][...,0],testMs[0][...,1]], Y)

Separated audio stream of the first speaker:

In [31]:
IPython.display.Audio(data=librosa.istft((X1test), hop_length=256), rate=22050)

Separated audio stream of the second speaker:

In [32]:
IPython.display.Audio(data=librosa.istft((X2test), hop_length=256), rate=22050)

The separation could be better, but some separation can already be heard. Also, to compare the results with that of Yu et al. 2016, the SDR value should be calculated for the separation.